# Climatological product

> Setelah membaca chapter ini, pembaca diharapkan dapat membuat produk klimatologis (peta spasial dan diagram windrose) berdasarkan wilayah pelayanan provinsi.

## Intro

Pada chapter sebelumnya, kita telah mencoba mengolah data multi dimensi menggunakan data sampel, mulai dari operasi dasar hingga visualisasi data pada peta spasial. Chapter ini akan menggabungkan beberapa chapter sebelumnya untuk membuat produk klimatologi spasial, area, dan titik. Terdapat tiga proses:
- load data: inacawo, inawaves, dan inaflows. Khusus untuk pelatihan ini, data sample tersedia selama tahun 2020-2025 untuk inawaves dan inaflows, serta 2025-sekarang untuk inacawo.
- preprocess data: agregrasi (perhitungan klimatologi).
- plot data: peta spasial dan wind rose.

Dalam hal ini, kita akan menggunakan template script yang telah disimpan dalam bentuk modul (`klimtool.py`) untuk mempermudah pemrosesan. 

Script `klimtool.py` berisi beberapa `class`, namun hanya `class` `klimtool` yang akan dipanggil di notebook ini. `class` `klimtool` memiliki method yang ditujukan sebagai data loader, preprocessor, hingga plotter. Untuk setiap langkah yang diselesaikan dengan memanggil modul tersebut, terdapat penjelasan singkat agar pembaca dapat memahami alur dari pemrosesan data.

## Load module & dataset

| Method | Deskripsi | Argumen mandatory |
| :- | :- | :- |
| `klimtool.open_inawaves` | Membuka dataset inawaves | - `tstart: datetime`,<br> - `tend: datetime`,<br> - `latlon: list[float]` | 
| `klimtool.open_inaflows` | Membuka dataset inaflows | - `tstart: datetime`,<br> - `tend: datetime`,<br> - `latlon: list[float]` | 
| `klimtool.open_inacawo` | Membuka dataset inacawo | - `tstart: datetime`,<br> - `tend: datetime`,<br> - `latlon: list[float]` | 

dimana
- `tstart`, `tend` -> masing-masing merupakan batas waktu awal dan akhir.
- `latlon` -> list berisi batas kiri dan kanan (longitude) serta bawah dan atas (latitude).

### Load module

In [1]:
from klimtool import klimtool # main module

import datetime

### Definisikan batas waktu dan area

In [2]:
timestart = datetime.datetime(2020,1,1)
timeend   = datetime.datetime(2025,5,1)
lonw = 95
lone = 145
latb = -13
latt = 10
latlon = [lonw, lone, latb, latt]

### Load Inawaves

In [3]:
%%time
dsinawaves = klimtool().open_inawaves(tstart=timestart, tend=timeend, latlon=latlon)
dsinawaves

/home/tyo/marinemet-training/2_/klimtool.py:351: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dset = xr.open_zarr(self.__INAWAVE_PATH__)


<class 'datetime.datetime'>
<class 'numpy.datetime64'>
CPU times: user 1.01 s, sys: 131 ms, total: 1.14 s
Wall time: 1.36 s


<xarray.Dataset> Size: 166GB
Dimensions:  (time: 8768, lat: 369, lon: 801)
Coordinates:
  * lat      (lat) float64 3kB -13.0 -12.94 -12.88 -12.81 ... 9.875 9.938 10.0
  * lon      (lon) float64 6kB 95.0 95.06 95.12 95.19 ... 144.9 144.9 145.0
  * time     (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
Data variables: (12/16)
    dir      (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    hmax     (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    hs       (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    lm       (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    pdi00    (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    pdi01    (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    ...       ...
    ptp00    (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    ptp01    (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    ptp02    (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    t01      (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    uwnd     (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
    vwnd     (time, lat, lon) float32 10GB dask.array<chunksize=(8, 369, 801), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    description:  InaWaves Model - Merge Hindcast, Reanalysis, and Forecast
    email:        produksi.maritim@bmkg.go.id
    institution:  BMKG - Center For Marine Meteorology
    source:       WaveWatch3 forced by GFS Model

### Load Inaflows

In [4]:
%%time
dsinaflows = klimtool().open_inaflows(tstart=timestart, tend=timeend, latlon=latlon)
dsinaflows

CPU times: user 119 ms, sys: 87.4 ms, total: 206 ms
Wall time: 471 ms


<xarray.Dataset> Size: 63GB
Dimensions:  (time: 8768, lat: 576, lon: 625)
Coordinates:
  * lat      (lat) float64 5kB -13.0 -12.96 -12.92 -12.88 ... 9.92 9.96 10.0
  * lon      (lon) float64 5kB 95.04 95.12 95.2 95.28 ... 144.8 144.9 145.0
  * time     (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
Data variables:
    S        (time, lat, lon) float32 13GB dask.array<chunksize=(8, 576, 625), meta=np.ndarray>
    T        (time, lat, lon) float32 13GB dask.array<chunksize=(8, 576, 625), meta=np.ndarray>
    u        (time, lat, lon) float32 13GB dask.array<chunksize=(8, 576, 625), meta=np.ndarray>
    v        (time, lat, lon) float32 13GB dask.array<chunksize=(8, 576, 625), meta=np.ndarray>
    zeta     (time, lat, lon) float32 13GB dask.array<chunksize=(8, 576, 625), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

### Load Inacawo

In [5]:
%%time
dsinacawo = klimtool().open_inacawo(tstart=timestart, tend=timeend, latlon=latlon)
dsinacawo

CPU times: user 5.17 s, sys: 140 ms, total: 5.31 s
Wall time: 5.33 s


<xarray.Dataset> Size: 122GB
Dimensions:  (time: 637, lat: 920, lon: 2001)
Coordinates:
  * lat      (lat) float64 7kB -13.0 -12.97 -12.95 -12.92 ... 9.925 9.95 9.975
  * lon      (lon) float64 16kB 95.0 95.02 95.05 95.08 ... 144.9 145.0 145.0
  * time     (time) datetime64[ns] 5kB 2025-04-01 ... 2025-04-29T12:00:00
Data variables: (12/27)
    S        (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    T        (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    ch       (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    cycle    (time) datetime64[ns] 5kB dask.array<chunksize=(2,), meta=np.ndarray>
    dir      (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    gust     (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    ...       ...
    uw       (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    uwnd     (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    v        (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    vw       (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    vwnd     (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
    zeta     (time, lat, lon) float32 5GB dask.array<chunksize=(2, 920, 2001), meta=np.ndarray>
Attributes:
    contact:      produksi.maritim@bmkg.go.id
    conventions:  CF-1.8
    description:  Indonesia Coupled Atmosphere Wave Ocean (InaCAWO)
    institution:  BMKG - Center For Marine Meteorology
    source:       InaCAWO - 3km High Resolution

## Klimatologi spasial

| Method | Deskripsi | Argument mandatory |
| :- | :- | :- |
| `klimtool.plot_map` | Memulai plotting untuk peta spasial | - `model:str`,<br> - `ds:xarray.Dataset`,<br> - `timefreq:str`,<br> - `var:str`,<br> - `area_type:str`,<br> - `area_name:str`,<br> - `map_title: str`,<br> - `out_dir:str` |

### Preprocessing

frekuensi waktu (`1H`: per1jam , `3H`: per3jam, `1D`: harian, `MS`: bulanan, `YS`: Tahunan). 

> ⚠️Perlu diperhatikan untuk frekuensi per1jam hanya berlaku untuk dataset inacawo⚠️

### Plot

## Klimatologi area dan titik

| Method | Deskripsi | Argument mandatory |
| :- | :- | :- |
| `klimtool.plot_rose` | Memulai plotting untuk wind dan wave rose | `model:str`, `ds:xarray.Dataset`, `timefreq:str`, `var:str`, `area_type:str`, `area_name:str`,  `map_title: str`, `out_dir:str` |

### Preprocessing

### Plot

In [7]:
%%time
from minio import Minio

client = Minio("172.17.0.1:9990",
    access_key="moqs1u5xKGk6tRx3GbyJ",
    secret_key="OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    secure=False
)
client.list_buckets()

CPU times: user 236 ms, sys: 23.4 ms, total: 260 ms
Wall time: 430 ms


[Bucket('bucket-test'), Bucket('zarr-data')]

In [16]:
for obj in client.list_objects("zarr-data"):
    print(obj.object_name)

demo-dataset.zarr/
mydata.zarr/


In [4]:
import zarr
import fsspec
import numpy as np
from zarr.storage import FSStore

# Konfigurasi koneksi ke MinIO
fs = fsspec.filesystem(
    "s3",
    key="moqs1u5xKGk6tRx3GbyJ",
    secret="OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    endpoint_url="http://172.17.0.1:9990",
    skip_instance_cache=True,
)

# Nama bucket dan path untuk menyimpan data
bucket_name = "zarr-data"
store_path = f"{bucket_name}/mydata.zarr"

# Pastikan bucket sudah ada
if not fs.exists(bucket_name):
    fs.mkdir(bucket_name)

# Membuat store menggunakan FsspecStore
store = FSStore(url=store_path, fs=fs)

# Membuat data dummy
data = np.random.rand(500, 500)

# Menyimpan data ke Zarr di MinIO
z = zarr.create(shape=data.shape, chunks=(100, 100), dtype="f4", store=store)
z[:] = data

print(f"Zarr array berhasil disimpan di MinIO pada {store_path}")

Zarr array berhasil disimpan di MinIO pada zarr-data/mydata.zarr


In [3]:
print(zarr.__version__)

2.18.4


In [15]:
import xarray as xr
import numpy as np
import s3fs

# Buat xarray Dataset dummy
ds = xr.Dataset(
    {
        "temperature": (("lat", "lon"), np.random.rand(50, 100)),
        "salinity": (("lat", "lon"), np.random.rand(50, 100))
    },
    coords={
        "lat": np.linspace(-90, 90, 50),
        "lon": np.linspace(0, 359, 100)
    }
)

# Setup koneksi ke MinIO via s3fs
fs = s3fs.S3FileSystem(
    key="moqs1u5xKGk6tRx3GbyJ",
    secret="OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    client_kwargs={"endpoint_url": "http://172.17.0.1:9990"}
)

# Path penyimpanan di MinIO
store = FSStore(
    url="zarr-data/demo-dataset.zarr", 
    fs=fs
)

# Simpan ke Zarr (overwrite jika sudah ada)
# ds.to_zarr(store=store, storage_options={"client_kwargs": {"endpoint_url": "http://172.17.0.1:9990"}}, mode="w")
ds.to_zarr(store=store, mode="w")

print("✅ Dataset berhasil disimpan ke MinIO sebagai Zarr.")


✅ Dataset berhasil disimpan ke MinIO sebagai Zarr.


In [18]:
import xarray as xr
import s3fs
from zarr.storage import FSStore

# Koneksi MinIO
fs = s3fs.S3FileSystem(
    key="moqs1u5xKGk6tRx3GbyJ",
    secret="OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    client_kwargs={"endpoint_url": "http://172.17.0.1:9990"}
)

# Path dataset Zarr di MinIO
store = FSStore("zarr-data/demo-dataset.zarr", fs=fs)

# Buka Zarr
ds_reload = xr.open_zarr(store=store, consolidated=True)

print("✅ Dataset berhasil dimuat:")
ds_reload

✅ Dataset berhasil dimuat:


<xarray.Dataset> Size: 81kB
Dimensions:      (lat: 50, lon: 100)
Coordinates:
  * lat          (lat) float64 400B -90.0 -86.33 -82.65 ... 82.65 86.33 90.0
  * lon          (lon) float64 800B 0.0 3.626 7.253 10.88 ... 351.7 355.4 359.0
Data variables:
    salinity     (lat, lon) float64 40kB dask.array<chunksize=(50, 100), meta=np.ndarray>
    temperature  (lat, lon) float64 40kB dask.array<chunksize=(50, 100), meta=np.ndarray>

In [21]:
ds_reload.nbytes/1000

81.2

In [2]:
import xarray as xr
import numpy as np
import s3fs
import zarr
from zarr.storage import FSStore

def update_zarr_forecast(ds_new, zarr_path, fs_kwargs, time_dim="time"):
    fs = s3fs.S3FileSystem(**fs_kwargs)
    store = FSStore(zarr_path, fs=fs)

    t_new = ds_new[time_dim].values

    if not fs.exists(zarr_path):
        print("Zarr belum ada, inisialisasi...")
        ds_new.to_zarr(store=store, mode="w")
        return

    print("Membuka Zarr existing...")
    root = zarr.open_group(store, mode="a")
    z_meta = zarr.open_group(store, mode="r")  # read metadata via xarray
    t_existing = z_meta[time_dim][:]
    
    overlap_mask = np.isin(t_new, t_existing)
    new_mask = ~overlap_mask
    print(f"  Replace: {overlap_mask.sum()} | Append: {new_mask.sum()}")

    for var in ds_new.data_vars:
        zarr_array = root[var]
        data_new = ds_new[var].values

        for i, t in enumerate(t_new):
            if overlap_mask[i]:
                idx_existing = np.where(t_existing == t)[0][0]
                zarr_array[idx_existing] = data_new[i]

    if new_mask.any():
        ds_append = ds_new.isel({time_dim: np.where(new_mask)[0]})
        append_len = ds_append.dims[time_dim]
        
        for var in ds_new.data_vars:
            zarr_array = root[var]
            old_shape = zarr_array.shape[0]
            new_shape = old_shape + append_len

            print(f"  Resize var {var}: {old_shape} → {new_shape}")
            zarr_array.resize(new_shape, axis=0)
            zarr_array[old_shape:new_shape] = ds_append[var].values

        zarr_time = root[time_dim]
        zarr_time.resize(new_shape)
        zarr_time[old_shape:new_shape] = ds_append[time_dim].values

    print("✅ Update Zarr selesai.")

In [7]:
fs_options = {
    "key": "moqs1u5xKGk6tRx3GbyJ",
    "secret": "OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    "client_kwargs": {
        "endpoint_url": "172.17.0.1:9990"
    }
}

update_zarr_forecast(ds_new, "zarr-data/model-archive.zarr", fs_options)

In [6]:
fs_options?

Type:        dict
String form: {'key': 'moqs1u5xKGk6tRx3GbyJ', 'secret': 'OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821', 'client_kwargs': {'endpoint_url': '172.17.0.1:9990'}}
Length:      3
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)